In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report,precision_score, recall_score

In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download stopwords and tokenizer
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meetd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\meetd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
data = pd.read_csv('reviews_dataset.csv')

In [7]:
data.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    30000 non-null  object
 1   brand                 30000 non-null  object
 2   categories            30000 non-null  object
 3   manufacturer          29859 non-null  object
 4   name                  30000 non-null  object
 5   reviews_date          29954 non-null  object
 6   reviews_didPurchase   15932 non-null  object
 7   reviews_doRecommend   27430 non-null  object
 8   reviews_rating        30000 non-null  int64 
 9   reviews_text          30000 non-null  object
 10  reviews_title         29810 non-null  object
 11  reviews_userCity      1929 non-null   object
 12  reviews_userProvince  170 non-null    object
 13  reviews_username      29937 non-null  object
 14  user_sentiment        29999 non-null  object
dtypes: int64(1), object(14)
memory usage

In [11]:
data.shape

(30000, 15)

In [13]:
data.isnull().sum()

id                          0
brand                       0
categories                  0
manufacturer              141
name                        0
reviews_date               46
reviews_didPurchase     14068
reviews_doRecommend      2570
reviews_rating              0
reviews_text                0
reviews_title             190
reviews_userCity        28071
reviews_userProvince    29830
reviews_username           63
user_sentiment              1
dtype: int64

In [15]:
data.shape

(30000, 15)

In [17]:
data = data.dropna(subset=['user_sentiment'])

In [19]:
data = data.drop(['id','brand','categories','manufacturer','name','reviews_date','reviews_didPurchase','reviews_doRecommend','reviews_rating','reviews_title','reviews_userCity','reviews_userProvince','reviews_username'],axis=1)

In [21]:
data.head()

,reviews_text,user_sentiment
0,i love this album. it's very good. more to the...,Positive
1,Good flavor. This review was collected as part...,Positive
2,Good flavor.,Positive
3,I read through the reviews on here before look...,Negative
4,My husband bought this gel for us. The gel cau...,Negative


In [23]:
data['user_sentiment'].value_counts()

user_sentiment
Positive    26632
Negative     3367
Name: count, dtype: int64

In [25]:
def clean_review(review):
    # 1. Convert to lowercase
    review = review.lower()
    
    # 2. Remove HTML tags
    review = re.sub(r'<.*?>', '', review)
    
    # 3. Remove special characters and numbers
    review = re.sub(r'[^a-z\s]', '', review)
    
    # 4. Tokenize the text
    tokens = word_tokenize(review)
    
    # 5. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # 6. Join tokens back into a cleaned string
    return ' '.join(tokens)


In [49]:

data['cleaned_reviews'] = data['reviews_text'].apply(clean_review)

# Display cleaned reviews
print(data[['reviews_text', 'cleaned_reviews']].head())


                                        reviews_text  \
0  i love this album. it's very good. more to the...   
1  Good flavor. This review was collected as part...   
2                                       Good flavor.   
3  I read through the reviews on here before look...   
4  My husband bought this gel for us. The gel cau...   

                                     cleaned_reviews  
0  love album good hip hop side current pop sound...  
1        good flavor review collected part promotion  
2                                        good flavor  
3  read reviews looking buying one couples lubric...  
4  husband bought gel us gel caused irritation fe...  


In [29]:
data = data.dropna(subset=['cleaned_reviews', 'user_sentiment'])

In [31]:
data['user_sentiment'] = data['user_sentiment'].map({'Positive': 1, 'Negative': 0})

In [33]:
print(data.shape)  # Check the number of rows and columns
print(data['user_sentiment'].isnull().sum())  # Confirm no NaN values remain

(29999, 3)
0


In [35]:
X = data['cleaned_reviews']  # Feature: text data
y = data['user_sentiment']  # Target: sentiment

In [37]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
vectorizer = CountVectorizer()  # Initialize CountVectorizer

# Fit and transform training data, transform test data
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [41]:
# Define a dictionary to store models and their names
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

In [43]:
# Store results
results = {}

In [45]:
# Iterate through models and train/test each one
for model_name, model in models.items():
    print(f"Training and testing {model_name}...")
    
    # Train the model
    model.fit(X_train_vec, y_train)
    
    # Predict on training data for overfitting check
    y_train_pred = model.predict(X_train_vec)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    
    # Predict on test data
    y_test_pred = model.predict(X_test_vec)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    classification_rep = classification_report(y_test, y_test_pred)
    
    # Check for overfitting or underfitting
    if train_accuracy > test_accuracy + 0.1:
        status = "Overfitting"
    elif train_accuracy < 0.7 and test_accuracy < 0.7:
        status = "Underfitting"
    else:
        status = "Well Generalized"

    # Store results
    results[model_name] = {
        "Training Accuracy": train_accuracy,
        "Testing Accuracy": test_accuracy,
        "Classification Report": classification_rep,
        "Status": status
    }

Training and testing Logistic Regression...
Training and testing Decision Tree...
Training and testing Random Forest...
Training and testing SVM...
Training and testing KNN...


In [47]:
# Print results
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    print(f"Training Accuracy: {metrics['Training Accuracy']:.2f}")
    print(f"Testing Accuracy: {metrics['Testing Accuracy']:.2f}")
    print(f"Status: {metrics['Status']}")
    print("Classification Report:")
    print(metrics['Classification Report'])


Model: Logistic Regression
Training Accuracy: 0.99
Testing Accuracy: 0.96
Status: Well Generalized
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.75      0.79       653
           1       0.97      0.98      0.98      5347

    accuracy                           0.96      6000
   macro avg       0.90      0.86      0.88      6000
weighted avg       0.95      0.96      0.95      6000


Model: Decision Tree
Training Accuracy: 1.00
Testing Accuracy: 0.94
Status: Well Generalized
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.73      0.74       653
           1       0.97      0.97      0.97      5347

    accuracy                           0.94      6000
   macro avg       0.86      0.85      0.85      6000
weighted avg       0.94      0.94      0.94      6000


Model: Random Forest
Training Accuracy: 1.00
Testing Accuracy: 0.94
Status: Well Generalized
Classificati